In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
from itertools import combinations
from statsmodels.tsa.stattools import adfuller

In [3]:
minimumAdv = 10

In [4]:
tpxData = pd.read_csv('TPX_prices.csv', index_col=0, parse_dates=True)
# tpxData

In [5]:
tpxData = tpxData.dropna(axis='columns')
# tpxData

In [6]:
tpxUniverseData = pd.read_excel('TPX Universe.xlsx')
# tpxUniverseData

In [7]:
tickersName = tpxData.columns

In [8]:
tpxUniverseData.tail()

,Ticker,Name,Market Cap,Avg of Daily Equity Traded Val over 3 Months,GICS Sector,GICS Ind Grp Name,GICS Ind Name,GICS SubInd Name
2134,7044 JP Equity,PIALA INC,0.012237,0.048263,Communication Services,Media & Entertainment,Media,Advertising
2135,3559 JP Equity,P-BAN.COM CORP,0.011896,0.014414,Information Technology,Technology Hardware & Equipment,"Electronic Equipment, Instruments & Components",Electronic Components
2136,3639 JP Equity,VOLTAGE INC,0.010484,0.230052,Communication Services,Media & Entertainment,Entertainment,Interactive Home Entertainment
2137,6029 JP Equity,ARTRA GROUP CORP,0.010471,0.048536,Health Care,Health Care Equipment & Services,Health Care Providers & Services,Health Care Services
2138,7615 JP Equity,YU-WA CREATION HOLDINGS CO L,0.010337,0.103814,Consumer Discretionary,Consumer Discretionary Distribution & Retail,Specialty Retail,Apparel Retail


In [9]:
for tickers in tickersName:
    if (tpxUniverseData.loc[tpxUniverseData['Ticker'] == tickers]['Avg of Daily Equity Traded Val over 3 Months'] < minimumAdv).bool():
        tpxData = tpxData.drop([tickers], axis=1)

/var/folders/h2/r7qn2m9n1zb6y_0q191gdqth0000gn/T/ipykernel_13129/4153440384.py:2: FutureWarning: Series.bool is now deprecated and will be removed in future version of pandas
  if (tpxUniverseData.loc[tpxUniverseData['Ticker'] == tickers]['Avg of Daily Equity Traded Val over 3 Months'] < minimumAdv).bool():
/var/folders/h2/r7qn2m9n1zb6y_0q191gdqth0000gn/T/ipykernel_13129/4153440384.py:2: FutureWarning: Series.bool is now deprecated and will be removed in future version of pandas
  if (tpxUniverseData.loc[tpxUniverseData['Ticker'] == tickers]['Avg of Daily Equity Traded Val over 3 Months'] < minimumAdv).bool():
/var/folders/h2/r7qn2m9n1zb6y_0q191gdqth0000gn/T/ipykernel_13129/4153440384.py:2: FutureWarning: Series.bool is now deprecated and will be removed in future version of pandas
  if (tpxUniverseData.loc[tpxUniverseData['Ticker'] == tickers]['Avg of Daily Equity Traded Val over 3 Months'] < minimumAdv).bool():
/var/folders/h2/r7qn2m9n1zb6y_0q191gdqth0000gn/T/ipykernel_13129/41534403

In [10]:
instrumentList = list(tpxData.columns)
pair_list = [('5401 JP Equity', '8604 JP Equity'), ('6273 JP Equity', '9984 JP Equity'), ('8053 JP Equity', '8058 JP Equity'), ('7733 JP Equity', '9613 JP Equity'), ('4684 JP Equity', '7832 JP Equity'), ('6762 JP Equity', '6857 JP Equity'), ('9020 JP Equity', '9022 JP Equity'), ('7267 JP Equity', '8306 JP Equity'), ('8308 JP Equity', '8802 JP Equity'), ('4901 JP Equity', '6702 JP Equity'), ('6503 JP Equity', '7269 JP Equity'), ('7267 JP Equity', '8801 JP Equity'), ('4519 JP Equity', '7532 JP Equity'), ('6988 JP Equity', '7267 JP Equity'), ('6326 JP Equity', '6954 JP Equity'), ('6752 JP Equity', '8604 JP Equity'), ('4901 JP Equity', '9613 JP Equity')]
len(pair_list)

17

In [11]:
pair_list

[('5401 JP Equity', '8604 JP Equity'),
 ('6273 JP Equity', '9984 JP Equity'),
 ('8053 JP Equity', '8058 JP Equity'),
 ('7733 JP Equity', '9613 JP Equity'),
 ('4684 JP Equity', '7832 JP Equity'),
 ('6762 JP Equity', '6857 JP Equity'),
 ('9020 JP Equity', '9022 JP Equity'),
 ('7267 JP Equity', '8306 JP Equity'),
 ('8308 JP Equity', '8802 JP Equity'),
 ('4901 JP Equity', '6702 JP Equity'),
 ('6503 JP Equity', '7269 JP Equity'),
 ('7267 JP Equity', '8801 JP Equity'),
 ('4519 JP Equity', '7532 JP Equity'),
 ('6988 JP Equity', '7267 JP Equity'),
 ('6326 JP Equity', '6954 JP Equity'),
 ('6752 JP Equity', '8604 JP Equity'),
 ('4901 JP Equity', '9613 JP Equity')]

In [12]:
# (tpxData.pct_change()+1).cumprod().plot(figsize=(25, 20), legend=False)

# plt.ylabel("Percentage Change")
# plt.show()

In [13]:
def findHedgeRatio(x, y):
    """
    Calculates the hedge ratio between two variables.

    Parameters:
    x: panda dataframe timeseries for instrument A
    y: panda dataframe timeseries for instrument B

    Returns:
    float: The hedge ratio.

    """
    model = sm.OLS(x, sm.add_constant(y)).fit()
    return model.params

In [14]:
# def findSpread(x, y):
#     """
#     Calculates the spread between two variables.

#     Parameters:
#     x: panda dataframe timeseries for instrument A
#     y: panda dataframe timeseries for instrument B

#     Returns:
#     panda dataframe: The spread between the two instruments.

#     """
#     hedgeRatio = findHedgeRatio(x, y)
#     spread = x - hedgeRatio * y
#     return spread

In [15]:
def ADFisStionaryTest(spread):
    """
    Augmented Dickey-Fuller test for stationarity.

    Parameters:
    spread: panda dataframe timeseries

    Returns:
    True if pair is stationary
    False if pair is not stationary
    """
    result = adfuller(spread, maxlag=1)
    # print('ADF Statistic: %f' % result[0])
    # print('p-value: %f' % result[1])
    # print('Critical Values:')
    # for key, value in result[4].items():
    #     print('\t%s: %.3f' % (key, value))
    if (result[0] < result[4]['1%']):
        return True
    else:
        return False

In [16]:
validPairsList = []

### Find valid pairs and its spread

In [17]:
pair = pair_list[0]
pair

('5401 JP Equity', '8604 JP Equity')

In [18]:

dfSpread = pd.DataFrame()
hedgeRatioParams = findHedgeRatio(tpxData[pair[0]], tpxData[pair[1]])
dfSpread['spread'] = tpxData[pair[0]] - hedgeRatioParams[1] * tpxData[pair[1]] - hedgeRatioParams[0]

/var/folders/h2/r7qn2m9n1zb6y_0q191gdqth0000gn/T/ipykernel_13129/3320111974.py:3: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  dfSpread['spread'] = tpxData[pair[0]] - hedgeRatioParams[1] * tpxData[pair[1]] - hedgeRatioParams[0]


In [19]:
# ### DO NOT REMOVE

validPairs = pd.DataFrame()
for pair in pair_list:
    dfSpread = pd.DataFrame()
    hedgeRatioParams = findHedgeRatio(tpxData[pair[0]], tpxData[pair[1]])
    dfSpread['spread'] = tpxData[pair[0]] - hedgeRatioParams[1] * tpxData[pair[1]] - hedgeRatioParams[0]
    adfResult = ADFisStionaryTest(dfSpread.iloc[:261]['spread'])
    validPairs[f'spread_{pair[0]}_{pair[1]}'] = dfSpread['spread']
    validPairsList.append(pair)

/var/folders/h2/r7qn2m9n1zb6y_0q191gdqth0000gn/T/ipykernel_13129/111423997.py:7: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  dfSpread['spread'] = tpxData[pair[0]] - hedgeRatioParams[1] * tpxData[pair[1]] - hedgeRatioParams[0]
/var/folders/h2/r7qn2m9n1zb6y_0q191gdqth0000gn/T/ipykernel_13129/111423997.py:7: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  dfSpread['spread'] = tpxData[pair[0]] - hedgeRatioParams[1] * tpxData[pair[1]] - hedgeRatioParams[0]
/var/folders/h2/r7qn2m9n1zb6y_0q191gdqth0000gn/T/ipykernel_13129/111423997.py:7: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future

In [20]:
validPairs

,spread_5401 JP Equity_8604 JP Equity,spread_6273 JP Equity_9984 JP Equity,spread_8053 JP Equity_8058 JP Equity,spread_7733 JP Equity_9613 JP Equity,spread_4684 JP Equity_7832 JP Equity,spread_6762 JP Equity_6857 JP Equity,spread_9020 JP Equity_9022 JP Equity,spread_7267 JP Equity_8306 JP Equity,spread_8308 JP Equity_8802 JP Equity,spread_4901 JP Equity_6702 JP Equity,spread_6503 JP Equity_7269 JP Equity,spread_7267 JP Equity_8801 JP Equity,spread_4519 JP Equity_7532 JP Equity,spread_6988 JP Equity_7267 JP Equity,spread_6326 JP Equity_6954 JP Equity,spread_6752 JP Equity_8604 JP Equity,spread_4901 JP Equity_9613 JP Equity
Date,,,,,,,,,,,,,,,,,
1/1/2013,54.852000,84.123220,-48.821833,-88.567163,478.697857,-1372.823966,92.111099,65.321051,-152.487540,-422.219533,-303.939690,106.694823,228.235715,-2828.200532,-501.109782,-612.675783,-98.971841
2/1/2013,54.852000,84.123220,-48.821833,-88.567163,478.697857,-1372.823966,92.111099,65.321051,-152.487540,-422.219533,-303.939690,106.694823,228.235715,-2828.200532,-501.109782,-612.675783,-98.971841
3/1/2013,54.852000,84.123220,-48.821833,-88.567163,478.697857,-1372.823966,92.111099,65.321051,-152.487540,-422.219533,-303.939690,106.694823,228.235715,-2828.200532,-501.109782,-612.675783,-98.971841
4/1/2013,59.631187,932.962521,-32.963438,-97.627231,457.921051,-1349.844618,87.874293,89.675191,-148.769895,-416.244517,-316.816182,140.616497,228.856264,-2781.288194,-505.260651,-616.177609,-92.477405
7/1/2013,132.761718,398.319726,-23.620957,-82.863401,449.577706,-1323.286299,101.193251,82.408533,-153.655284,-401.626364,-307.755527,126.098205,228.976078,-2791.946086,-465.365413,-595.270586,-81.196384
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27/5/2024,-212.964671,1416.144321,115.088505,-219.981606,-799.459386,360.159567,-69.970327,-89.811513,407.791004,465.433356,711.312149,127.430227,-2001.370596,1362.502720,264.516152,-174.306710,425.357954
28/5/2024,-205.204120,-671.034485,69.041304,-225.435968,-199.441200,519.242173,-71.267722,-114.092806,441.341668,565.338984,729.471175,146.893163,-2073.169026,1275.144192,264.903529,-159.430137,533.916452
29/5/2024,-219.190378,-4372.249273,72.243596,-213.013673,-211.240768,364.858229,-89.677370,-129.880317,428.139011,547.382120,616.692335,147.264640,-2084.772755,1210.450881,226.640566,-167.793718,516.142253


In [21]:
valid = validPairs
# valid = pd.read_csv('validPairs3.csv', index_col=0, parse_dates=True)
valid

,spread_5401 JP Equity_8604 JP Equity,spread_6273 JP Equity_9984 JP Equity,spread_8053 JP Equity_8058 JP Equity,spread_7733 JP Equity_9613 JP Equity,spread_4684 JP Equity_7832 JP Equity,spread_6762 JP Equity_6857 JP Equity,spread_9020 JP Equity_9022 JP Equity,spread_7267 JP Equity_8306 JP Equity,spread_8308 JP Equity_8802 JP Equity,spread_4901 JP Equity_6702 JP Equity,spread_6503 JP Equity_7269 JP Equity,spread_7267 JP Equity_8801 JP Equity,spread_4519 JP Equity_7532 JP Equity,spread_6988 JP Equity_7267 JP Equity,spread_6326 JP Equity_6954 JP Equity,spread_6752 JP Equity_8604 JP Equity,spread_4901 JP Equity_9613 JP Equity
Date,,,,,,,,,,,,,,,,,
1/1/2013,54.852000,84.123220,-48.821833,-88.567163,478.697857,-1372.823966,92.111099,65.321051,-152.487540,-422.219533,-303.939690,106.694823,228.235715,-2828.200532,-501.109782,-612.675783,-98.971841
2/1/2013,54.852000,84.123220,-48.821833,-88.567163,478.697857,-1372.823966,92.111099,65.321051,-152.487540,-422.219533,-303.939690,106.694823,228.235715,-2828.200532,-501.109782,-612.675783,-98.971841
3/1/2013,54.852000,84.123220,-48.821833,-88.567163,478.697857,-1372.823966,92.111099,65.321051,-152.487540,-422.219533,-303.939690,106.694823,228.235715,-2828.200532,-501.109782,-612.675783,-98.971841
4/1/2013,59.631187,932.962521,-32.963438,-97.627231,457.921051,-1349.844618,87.874293,89.675191,-148.769895,-416.244517,-316.816182,140.616497,228.856264,-2781.288194,-505.260651,-616.177609,-92.477405
7/1/2013,132.761718,398.319726,-23.620957,-82.863401,449.577706,-1323.286299,101.193251,82.408533,-153.655284,-401.626364,-307.755527,126.098205,228.976078,-2791.946086,-465.365413,-595.270586,-81.196384
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27/5/2024,-212.964671,1416.144321,115.088505,-219.981606,-799.459386,360.159567,-69.970327,-89.811513,407.791004,465.433356,711.312149,127.430227,-2001.370596,1362.502720,264.516152,-174.306710,425.357954
28/5/2024,-205.204120,-671.034485,69.041304,-225.435968,-199.441200,519.242173,-71.267722,-114.092806,441.341668,565.338984,729.471175,146.893163,-2073.169026,1275.144192,264.903529,-159.430137,533.916452
29/5/2024,-219.190378,-4372.249273,72.243596,-213.013673,-211.240768,364.858229,-89.677370,-129.880317,428.139011,547.382120,616.692335,147.264640,-2084.772755,1210.450881,226.640566,-167.793718,516.142253


In [22]:
validPairs.to_csv("validPairs5.csv")

### Strategy

Enter when price goes above 1 std.

Cut loss when price goes above 1.5 std.

Take profit when spread goes to 0 or change sign.


In [23]:
if len(validPairsList) == 0:
    validPairsList = []

    for spread_key in valid.keys():
        # Extract pairs by splitting the string, assuming format 'spread_X_Y'
        pair = spread_key.replace('spread_', '').split('_')
        validPairsList.append(tuple(pair))

In [24]:
rollingWindow = 262
cutLossSd = 2

In [25]:
pairsOutcome = {}

In [26]:
clustering_validPairsList = [('5401 JP Equity', '8604 JP Equity'), ('6273 JP Equity', '9984 JP Equity'), ('8053 JP Equity', '8058 JP Equity'), ('7733 JP Equity', '9613 JP Equity'), ('4684 JP Equity', '7832 JP Equity'), ('6762 JP Equity', '6857 JP Equity'), ('9020 JP Equity', '9022 JP Equity'), ('7267 JP Equity', '8306 JP Equity'), ('8308 JP Equity', '8802 JP Equity'), ('4901 JP Equity', '6702 JP Equity'), ('6503 JP Equity', '7269 JP Equity'), ('7267 JP Equity', '8801 JP Equity'), ('4519 JP Equity', '7532 JP Equity'), ('6988 JP Equity', '7267 JP Equity'), ('6326 JP Equity', '6954 JP Equity'), ('6752 JP Equity', '8604 JP Equity'), ('4901 JP Equity', '9613 JP Equity')]

In [27]:
return_df = (tpxData / tpxData.shift(1)) - 1

for pair in clustering_validPairsList:
    df = pd.DataFrame()

    #Calculate Standard Deviations
    df['spread'] = valid[f'spread_{pair[0]}_{pair[1]}']
    df['mid'] =  df['spread'].rolling(rollingWindow).mean()
    df['1sd high'] = df['spread'].rolling(rollingWindow).mean() + df['spread'].rolling(rollingWindow).std()
    df['1sd low'] = df['spread'].rolling(rollingWindow).mean() - df['spread'].rolling(rollingWindow).std()
    df['2sd high'] = df['spread'].rolling(rollingWindow).mean() + df['spread'].rolling(rollingWindow).std() * cutLossSd
    df['2sd low'] = df['spread'].rolling(rollingWindow).mean() - df['spread'].rolling(rollingWindow).std() * cutLossSd
    df['position'] = 0

    df.loc[(df['spread'] > df['1sd high']) & (df['spread'] < df['2sd high']), 'position'] = -1
    df.loc[(df['spread']< df['1sd low']) & (df['spread'] > df['2sd low']), 'position'] = 1
    ## changes high to low in line 16

    #Calculate PnL
    df[f'{pair[0]} position'] = df['position']
    df[f'{pair[1]} position'] = df['position'] * -1
    df['dailypnl'] = df[f'{pair[1]} position']*return_df[f'{pair[1]}'].shift(-1) + df[f'{pair[0]} position']*return_df[f'{pair[0]}'].shift(-1)
    df['cumpnl'] = df['dailypnl'].cumsum()

    pairsOutcome[f'{pair[0]} {pair[1]}'] = df

    ## Can we use ML or NN to size the trade
    ## use ML to time the trade instead of only using SD


In [28]:
# Find the key with the greatest value of cumpnl[-2] tommorrow (dailypnl) return is in today row
best_key = max(
    pairsOutcome,
    key=lambda k: pairsOutcome[k].cumpnl.iloc[-2]  # Access cumpnl[-2] safely
)

print("Key with the greatest value:", best_key)
print("Greatest value:", pairsOutcome[best_key].cumpnl.iloc[-2])

Key with the greatest value: 6503 JP Equity 7269 JP Equity
Greatest value: 1.332416127365959


In [29]:
import pickle

# Save the dictionary to a file
with open('pairsOutcome.pkl', 'wb') as file:
    pickle.dump(pairsOutcome, file)

print("Dictionary saved to pairsOutcome.pkl")

Dictionary saved to pairsOutcome.pkl


In [30]:
pairsOutcome.items()

dict_items([('5401 JP Equity 8604 JP Equity',                spread         mid     1sd high     1sd low     2sd high  \
Date                                                                      
1/1/2013    54.852000         NaN          NaN         NaN          NaN   
2/1/2013    54.852000         NaN          NaN         NaN          NaN   
3/1/2013    54.852000         NaN          NaN         NaN          NaN   
4/1/2013    59.631187         NaN          NaN         NaN          NaN   
7/1/2013   132.761718         NaN          NaN         NaN          NaN   
...               ...         ...          ...         ...          ...   
27/5/2024 -212.964671  640.517283  1060.310340  220.724226  1480.103398   
28/5/2024 -205.204120  636.742095  1059.676537  213.807653  1482.610979   
29/5/2024 -219.190378  632.926338  1059.056222  206.796454  1485.186105   
30/5/2024 -157.208034  629.200068  1057.956382  200.443753  1486.712697   
31/5/2024 -216.067718  625.625385  1057.510410  193.74

In [31]:
tpxData['8604 JP Equity']

Date
1/1/2013     503.0
2/1/2013     503.0
3/1/2013     503.0
4/1/2013     524.0
7/1/2013     498.0
             ...  
27/5/2024    940.7
28/5/2024    936.3
29/5/2024    928.2
30/5/2024    909.5
31/5/2024    950.5
Name: 8604 JP Equity, Length: 2979, dtype: float64